# O Problema
Sliding Puzzle - Bloco Deslizante

In [1]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

In [2]:
import numpy as np

class Puzzle:
  def __init__(self, board=None, min_moves=5, max_moves=20):  
    if board is None:
      self.board = np.array([[1,2,3],[4,5,6],[7,8,-1]])
      self.empty = (2,2)

      self.shuffle(min_moves, max_moves)
    else:
      self.board = board
      e = np.where(board == -1)
      self.empty = (e[0][0], e[1][0])


  def encode(self):
    return np.array2string(self.board.flatten(), separator='')
        
  def print_board(self):
    print(self.board)
        
  def get_allowed_moves(self):
    allowed_moves = []
    
    allowed_moves += self.get_vertical_moves()
    allowed_moves += self.get_horizontal_moves()

    return allowed_moves

    
  def get_vertical_moves(self):
    allowed_moves = []
    if self.empty[0] > 0:
      allowed_moves.append((self.empty[0] - 1, self.empty[1])) # down
    if self.empty[0] < len(self.board) - 1:
      allowed_moves.append((self.empty[0] + 1, self.empty[1])) # up
        
    return allowed_moves
          
  def get_horizontal_moves(self):
    allowed_moves = []
    if self.empty[1] > 0:
      allowed_moves.append((self.empty[0], self.empty[1] - 1)) # right
    if self.empty[1] < len(self.board) - 1:
      allowed_moves.append((self.empty[0], self.empty[1] + 1)) # left
          
    return allowed_moves
  

    
  def move(self, piece_to_move):
    self.board[self.empty] = self.board[piece_to_move]
    self.board[piece_to_move] = -1
      
    self.empty = piece_to_move
        
    
    
  def check_puzzle(self):
    if np.array_equal(self.board, np.array([[1,2,3],[4,5,6],[7,8,-1]])): 
      return True
    
    return False


                
                
  def shuffle(self, min_moves, max_moves):
    rng = np.random.default_rng()
    num_of_moves = rng.integers(min_moves, max_moves)

    for i in range(num_of_moves):
      allowed_moves = self.get_allowed_moves()
      move = rng.integers(0, len(allowed_moves))

      self.move((allowed_moves[move]))


  def distance_to_solution(self):
    return np.sum(self.board != np.array([[1,2,3],[4,5,6],[7,8,-1]]))


In [3]:
class Tree:
  def __init__(self, movement, puzzle, parent=None):
    self.movement = movement
    self.puzzle = puzzle
    self.children = puzzle.get_allowed_moves()
    self.parent = parent
    
    if parent is None:
      self.g = 0
    else:
      self.g = parent.g + 1

    self.h = puzzle.distance_to_solution()
    self.f = self.g + self.h

    self.visited = False

  def create_child(self, i): # cria o filho i (adiciona o puzzle, executa o movimento, cria a árvore)
    p = Puzzle(self.puzzle.board.copy())
    p.move(self.children[i])

    self.children[i] = Tree(self.children[i], p, self)

# Resolver o quebra-cabeças usando Buscas

## Gerar o caminho

In [4]:
from collections import deque

In [5]:
def create_path(node):
  q = deque()
  q.append(node)

  while node.parent is not None:
    node = node.parent
    q.appendleft(node)

  return q

## Busca usando A*

In [6]:
def a_star_search(tree):
  open = []
  closed = set()

  open.append(tree)

  while open:
    open.sort(key=lambda x: x.f, reverse=True)

    current_node = open.pop()

    closed.add(current_node.puzzle.encode())

    if current_node.puzzle.check_puzzle():
      return current_node, create_path(current_node)
    

    for i in range(len(current_node.children)):
      if type(current_node.children[i]) != Tree:
        current_node.create_child(i)
      c = current_node.children[i]

      if c.puzzle.encode() in closed:
        continue

      for i, n in enumerate(open):
        if n.puzzle.encode() == c.puzzle.encode():
          if c.g < n.g:
            open[i] = c

          break

      open.append(c)

In [7]:
board = np.array([[1,2,3],[4,5,6],[7,8,-1]])

p = Puzzle(board=board)
t = Tree((-1,-1), p)

p.print_board()

n, path = a_star_search(t)

print("\nTamanho da solução: ", len(path))

for p in path:
  print("\nPeça movida: ", p.movement)
  p.puzzle.print_board()

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8 -1]]

Tamanho da solução:  1

Peça movida:  (-1, -1)
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8 -1]]


In [8]:
board = np.array([[1,-1,3],[4,2,5],[7,8,6]])

p = Puzzle(board=board)
t = Tree((-1,-1), p)

p.print_board()

n, path = a_star_search(t)

print("\nTamanho da solução: ", len(path))

for p in path:
  print("\nPeça movida: ", p.movement)
  p.puzzle.print_board()

[[ 1 -1  3]
 [ 4  2  5]
 [ 7  8  6]]

Tamanho da solução:  4

Peça movida:  (-1, -1)
[[ 1 -1  3]
 [ 4  2  5]
 [ 7  8  6]]

Peça movida:  (1, 1)
[[ 1  2  3]
 [ 4 -1  5]
 [ 7  8  6]]

Peça movida:  (1, 2)
[[ 1  2  3]
 [ 4  5 -1]
 [ 7  8  6]]

Peça movida:  (2, 2)
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8 -1]]


In [9]:
p = Puzzle(min_moves=25, max_moves=30)
t = Tree((-1,-1), p)

p.print_board()

n, path = a_star_search(t)

[[ 1  5  2]
 [-1  4  3]
 [ 7  8  6]]


In [10]:
print("Tamanho da solução: ", len(path))

for p in path:
  print("\nPeça movida: ", p.movement)
  p.puzzle.print_board()

Tamanho da solução:  6

Peça movida:  (-1, -1)
[[ 1  5  2]
 [-1  4  3]
 [ 7  8  6]]

Peça movida:  (1, 1)
[[ 1  5  2]
 [ 4 -1  3]
 [ 7  8  6]]

Peça movida:  (0, 1)
[[ 1 -1  2]
 [ 4  5  3]
 [ 7  8  6]]

Peça movida:  (0, 2)
[[ 1  2 -1]
 [ 4  5  3]
 [ 7  8  6]]

Peça movida:  (1, 2)
[[ 1  2  3]
 [ 4  5 -1]
 [ 7  8  6]]

Peça movida:  (2, 2)
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8 -1]]


## Discusta sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento



Consumo de memória: O algoritmo A* consegue obter a solução com consumo de memória geralmente menor que a busca em largura/em profundidade.

Embora precise armazenar todos os vizinhos dos nodos que já foram abertos na memória, ainda assim geralmente possui menor custo de memória quando comparado com a busca em profundidade, pois encontra a solução mais rasa, ao invés de buscar muito profundamente na árvore.

Comparando com a busca em largura, como ambas buscas vão até a mesma profundidade na árvore (onde se encontra a solução ótima), o custo é menor, porque somente os filhos dos nodos mais próximos da solução acabam sendo abertos, e não todos em cada profundidade.

Processamento: O A* tem um desempenho muito melhor que a busca em profundidade (na maioria dos casos), porque expande menos nodos que se afastam da solução, e acaba não indo para caminhos extremamente longos de movimentos aleatórios.

Comparado com a busca em largura, o A* deveria executar mais rapidamente, pelo mesmo motivo do custo de memória...

Porém, principalmente nas buscas em que é necessário piorar o estado muitas vezes para alcançar o resultado, teve desempenho pior que busca em largura. Possivelmente outra forma de calcular H resolveria.